## Every Dream v2 RunPod Setup

[General Instructions](https://github.com/victorchall/EveryDream2trainer/blob/main/README.md)

If you can sign up for Runpod here (shameless referral link): [Runpod](https://runpod.io/?ref=oko38cd0)

If you are confused by the wall of text, join the discord here: [EveryDream Discord](https://discord.gg/uheqxU6sXN)

### Requirements
Select the `RunPod Stable Diffusion v2.1` template. The `RunPod PyTorch` template does not work due to an old version of Python. 

#### Storage
Remember, on RunPod time is more expensive than storage. 

Which is good, because running a lot of experiments can generate a lot of data. Not having the right save points to recover quickly from inevitable mistakes will cost you a lot of time.

When in doubt, give yourself ~125GB of Runpod **Volume** storage.

#### Preparation
You will want to have your data prepared before starting, and have a rough training plan in mind. Don't waste rental fees if you're not fully prepared to start training.  

Your files should be captioned before you start with either the caption as the filename or in text files for each image alongside the image files.  See [DATA.md](https://github.com/victorchall/EveryDream2trainer/blob/main/doc/DATA.md) for more details. Tools are available to automatically caption your files.

In [ ]:
# When running on a pod designed for Automatic 1111 
# we need to kill the webui process to free up mem for training
!ps x | grep -E "(relauncher|webui)" | grep -v "grep" | awk '{print $1}' | xargs kill $1

# check system resources, make sure your GPU actually has 24GB
# You should see something like "0 MB / 24576 MB" in the middle of the printout
# if you see 0 MB / 22000 MB pick a beefier instance...
!grep Swap /proc/meminfo
!swapon -s
!nvidia-smi

In [ ]:
import os

%cd /workspace
if not os.path.exists("EveryDream2trainer"):
    !git clone https://github.com/victorchall/EveryDream2trainer

%cd EveryDream2trainer
!python utils/get_yamls.py

# Upload training files

Ues the navigation on the left to open the ** "workspace / EveryDream2trainer / input"** and upload your training files using the **up arrow button** above the file explorer, or by dragging and dropping the files from your local machine onto the file explorer.

If you have many training files, or nested folders of training data, create a zip archive of your training data, upload this file to the input folder, then right click on the zip file and select "Extract Archive".

**While your training data is uploading, feel free to install the dependencies below**

## Install dependencies

**This will take up to 15 minutes (if building xformers).  Wait until it says "DONE" to move on.** 
You can ignore "warnings."

In [ ]:
!python -m pip install --upgrade pip

!pip install requests==2.25.1
!pip install -U -I torch==1.13.1+cu116 torchvision==0.14.1+cu116 --extra-index-url "https://download.pytorch.org/whl/cu116"
!pip install transformers==4.25.1
!pip install -U diffusers[torch]==0.10.2

!pip install pynvml==11.4.1
!pip install bitsandbytes==0.35.0
!pip install ftfy==6.1.1
!pip install aiohttp==3.8.3
!pip install "tensorboard>=2.11.0"
!pip install protobuf==3.20.2
!pip install wandb==0.13.6
!pip install colorama==0.4.6
!pip install -U triton
!pip install -U ninja

from subprocess import getoutput
s = getoutput('nvidia-smi')

if "A100" in s:
    print("Detected A100, installing stable xformers")
    !pip install -U xformers
else:
    # A5000/3090/4090 support requires us to build xformers ourselves for now
    print("Building xformers for SM86")
    !apt-get update && apt-get install -y gcc g++
    !export TORCH_CUDA_ARCH_LIST=8.6 && pip install git+https://github.com/facebookresearch/xformers.git@48a77cc#egg=xformers

print("DONE")


# Optional - Configure sample prompts
You can set your own sample prompts by adding them, one line at a time, to sample_prompts.txt.

Keep in mind a longer list of prompts will take longer to generate. You may also want to adjust you sample_steps in the training notebook to a different value to get samples left often. This is probably a good idea when training a larger dataset that you know will take longer to train, where more frequent samples will not help you.

## Now that dependencies are installed, ready to move on!

## Log into huggingface
Run the cell below and paste your token into the prompt.  You can get your token from your [huggingface account page](https://huggingface.co/settings/tokens).

The token will not show on the screen, just press enter after you paste it.

Then run the following cell to download the base checkpoint (may take a minute).

In [ ]:
from huggingface_hub import notebook_login, hf_hub_download
import os
notebook_login()

In [ ]:
repo="panopstor/EveryDream"
ckpt_file="sd_v1-5_vae.ckpt"

print(f"Downloading {ckpt_file} from {repo}")
downloaded_model_path = hf_hub_download(repo, ckpt_file, cache_dir="/workspace/hfcache")
ckpt_name = os.path.splitext(os.path.basename(downloaded_model_path))[0]
print(f"Downloaded {ckpt_name} to {downloaded_model_path}")

if not os.path.exists(f"ckpt_cache/{ckpt_name}"):
    print(f"Converting {ckpt_name} to Diffusers format")
    !python utils/convert_original_stable_diffusion_to_diffusers.py --scheduler_type ddim \
    --original_config_file v1-inference.yaml \
    --image_size 512 \
    --checkpoint_path "{downloaded_model_path}" \
    --prediction_type epsilon \
    --upcast_attn False \
    --dump_path "ckpt_cache/{ckpt_name}"


print("DONE")

# Start Training
Naming your project will help you track what the heck you're doing when you're floating in checkpoint files later.

You may wish to consider adding "sd1" or "sd2v" or similar to remember what the base was, as you'll also have to tell your inference app what you were using, as its difficult for programs to know what inference YAML to use automatically. For instance, Automatic1111 webui requires you to copy the v2 inference YAML and rename it to match your checkpoint name so it knows how to load the file, tough it assumes SD 1.x compatible. Something to keep in mind if you start training on SD2.1.

`max_epochs`, `sample_steps`, and `save_every_n_epochs` should be tuned to your dataset. I like to generate one or two sets of samples per save, and aim for 5 (give or take 2) saved checkpoints.

Next cell runs training. This will take a while depending on your number of images, repeats, and max_epochs.

You can watch for test images in the logs folder.

In [ ]:
!python train.py --project_name "ft_v1a_512_15e7" \
--resume_ckpt "{ckpt_name}" \
--data_root "input" \
--resolution 512 \
--batch_size 4 \
--max_epochs 200 \
--save_every_n_epochs 25 \
--lr 1.5e-6 \
--lr_scheduler constant \
--sample_steps 50 \
--useadam8bit \
--save_full_precision \
--shuffle_tags \
--amp \
--write_schedule

## You can chain togther a tuning schedule using `--resume_ckpt findlast`

In [ ]:
!python train.py --project_name "ft_v1b_512_07e7" \
--resume_ckpt findlast \
--data_root "input" \
--resolution 512 \
--batch_size 4 \
--max_epochs 50 \
--save_every_n_epochs 25 \
--lr 0.7e-6 \
--lr_scheduler constant \
--sample_steps 50 \
--useadam8bit \
--save_full_precision \
--shuffle_tags \
--amp \
--write_schedule


# HuggingFace upload
Use the cell below to upload your checkpoint to your personal HuggingFace account if you want instead of manually downloading. You should already be authorized to Huggingface by token if you used the download/token cells above.

Make sure to fill in the three fields at the top. This will only upload one file at a time, so you will need to run the cell below for each file you want to upload.

* You can get your account name from your [HuggingFace account page](https://huggingface.co/settings/account). Look for your "username" field and paste it below.

* You only need to setup a repository one time.  You can create it here: [Create New HF Model](https://huggingface.co/new)  Make sure you write down the repo name you make for future use.  You can reuse it later.

* You need to type the name of the ckpts one at a time in the cell below, find them in the left file drawer of your Runpod/Vast/Colab.

In [ ]:
#list ckpts in root that are ready for download
import glob

for f in glob.glob("*.ckpt"):
    print(f)

In [ ]:
!huggingface-cli lfs-enable-largefiles
# fill in these three fields:
hfusername = "MyHfUser"
reponame = "MyRepo"
ckpt_name = "ft_v1b_512_15e7-ep200-gs02500.ckpt"


target_name = ckpt_name.replace('-','').replace('=','')
import os
os.rename(ckpt_name,target_name)
repo_id=f"{hfusername}/{reponame}"
print(f"uploading to HF: {repo_id}/{ckpt_name}")
print("this make take a while...")

from huggingface_hub import HfApi
api = HfApi()
response = api.upload_file(
    path_or_fileobj=target_name,
    path_in_repo=target_name,
    repo_id=repo_id,
    repo_type=None,
    create_pr=1,
)
print(response)
print(finish_msg)
print("go to your repo and accept the PR this created to see your file")